In [3]:
import pulp
import pandas as pd
import math


df = pd.read_excel("mansei.xlsx")
#従業員リスト
Emp = df.iloc[:,0].values.tolist()
df = df.fillna("br") #データ穴埋め
        

# パラメータ設定
start = 10 
last = 22 
num_Emp = len(Emp)
min_shifts_per_nurse = 1
max_shifts_per_nurse = 5
demand = [1, 2, 3, 3, 3, 2, 2, 3, 4, 4, 3, 3, 3]
num_days = len(demand)
day = 1

min_time = 3  #入れる最低時間
max_consecutive_workdays = 5  # 連続して働ける最大日数
min_consecutive_offdays = 2  # 連続して休む最少日数

# 変数定義

x = {}
y = {}
z = {}
for i in range(num_Emp):
    z[(i)] = pulp.LpVariable(f"z({i})", 0, 1, pulp.LpBinary)
    for t in range(start,last+1):
        x[(i, t)] = pulp.LpVariable(f"x({i},{t})", 0, 1, pulp.LpBinary)
        y[(i, t)] = pulp.LpVariable(f"y({i},{t})", 0, 1, pulp.LpBinary)


# 問題定義
prob = pulp.LpProblem("NurseSchedulingWithRest", pulp.LpMinimize)

# 制約の追加
#1時間ごとの働く従業員総数
for t in range(start,last+1):
    prob += pulp.lpSum([x[(i,t)] for i in range(num_Emp)]) == demand[t - start]

"""#一人が入れる最低時間と最高時間
for i in range(num_Emp):
    prob += pulp.lpSum([x[(i, j)] for j in range(start,last+1)]) >= min_shifts_per_nurse
    prob += pulp.lpSum([x[(i, j)] for j in range(start,last+1)]) <= max_shifts_per_nurse
"""
"""

    # 連続して働く時間数の制約
    for j in range(num_days - max_consecutive_workdays + 1):
        prob += pulp.lpSum([x[(i, j+k)] for k in range(max_consecutive_workdays)]) <= max_consecutive_workdays - 1

    # 連続して休む時間数の制約
    for j in range(num_days - min_consecutive_offdays + 1):
        prob += pulp.lpSum([1 - x[(i, j+k)] for k in range(min_consecutive_offdays)]) <= min_consecutive_offdays - 2
"""
#1勤務 min_time 時間以上 or 0時間
for i in range(num_Emp):
    time = 1
    while (min_time > time):
        #prob += pulp.lpSum([x[(i,t)]] for t in range(start ,last+1)) != time
        time += 1

for i in range(num_Emp):
    prob += pulp.lpSum([x[(i,t)]] for t in range(start ,last+1)) >= z[(i)]
    #prob += z[(i)] == int(math.ceil(a/100))
    #prob += (1 - math.ceil(a/100))*100 + pulp.lpSum([x[(i,t)]] for t in range(start ,last+1)) >= 3
    prob += (1-z[(i)])*100 + (pulp.lpSum([x[(i,t)]] for t in range(start ,last+1))) >= 3


#希望時間を守る
for i in range(0,num_Emp):
    if df.iloc[i,day] == "br": #休みの時
        for t in range(start,last+1):
            prob += pulp.lpSum(x[(i,t)]) == 0
    elif df.iloc[i,day] == "all": #いつでも
        for t in range(start,last+1):
            pass
            #prob += pulp.lpSum(x[(i,t)]) == 1
    else:  #時間指定
        a = df.iloc[i,day].split(("~"))
        a = [int(i) for i in a]
        for t in range(start,last+1):
            if a[0] <= t and t <= a[1]:
                pass
                #prob += pulp.lpSum(x[(i,t)]) == 1
            else:
                prob += pulp.lpSum(x[(i,t)]) == 0
    
# yを働いていない時間を１として設定
for i in range(num_Emp):
    for t in range(start,last+1):
        prob += y[(i,t)] == 1 - x[(i, t)]

#休みと休みの間を出勤にしない
for i in range(num_Emp):
    for t in range(start+1,last):
        prob += y[(i,t-1)] + x[(i,t)] + y[(i,t+1)] <= 2



#目的関数
prob += pulp.lpSum([-z[(i)]] for i in range(num_Emp))

# 問題の解決
sol = prob.solve()

# 結果の出力
for i in range(num_Emp):
    print(f"Emp {i+1}: ", end="")
    for j in range(start,last+1):
        if pulp.value(x[(i, j)]) == 1:
            print(j, end=" ")
    print()

print("Status:",pulp.LpStatus[sol])

Emp 1: 
Emp 2: 
Emp 3: 
Emp 4: 
Emp 5: 
Emp 6: 
Emp 7: 
Emp 8: 
Emp 9: 
Emp 10: 
Emp 11: 
Emp 12: 
Emp 13: 
Emp 14: 
Emp 15: 
Emp 16: 
Emp 17: 
Emp 18: 
Emp 19: 
Emp 20: 


AttributeError: 'generator' object has no attribute 'value'

In [3]:
# 結果の出力
for i in range(num_Emp):
    print(f"Emp {i+1}: ", end="")
    for j in range(start,last+1):
        if pulp.value(x[(i, j)]) == 1:
            print(x[i,j])
            print(pulp.value(x[(i, j)]), end=" ")
    print()

Emp 1: 
Emp 2: 
Emp 3: x(2,10)
1.0 x(2,11)
1.0 x(2,12)
1.0 
Emp 4: 
Emp 5: x(4,13)
1.0 x(4,14)
1.0 x(4,15)
1.0 x(4,17)
1.0 x(4,18)
1.0 x(4,19)
1.0 x(4,20)
1.0 x(4,21)
1.0 x(4,22)
1.0 
Emp 6: 
Emp 7: x(6,11)
1.0 x(6,12)
1.0 x(6,13)
1.0 x(6,14)
1.0 x(6,16)
1.0 x(6,17)
1.0 x(6,20)
1.0 x(6,21)
1.0 x(6,22)
1.0 
Emp 8: 
Emp 9: 
Emp 10: x(9,17)
1.0 x(9,18)
1.0 x(9,19)
1.0 x(9,20)
1.0 
Emp 11: 
Emp 12: 
Emp 13: 
Emp 14: x(13,18)
1.0 x(13,19)
1.0 x(13,21)
1.0 
Emp 15: 
Emp 16: 
Emp 17: x(16,12)
1.0 x(16,13)
1.0 x(16,14)
1.0 x(16,15)
1.0 x(16,16)
1.0 
Emp 18: 
Emp 19: x(18,18)
1.0 x(18,19)
1.0 x(18,22)
1.0 
Emp 20: 
